##### Trial and error for dashboard 
__Larissa Voshol__  
__11/12/2023__  
__BHVF2__  

In [1]:
import numpy as np
import pandas as pd
import panel as pn
import bokeh as bk
import param

pn.extension()


In [2]:
#import data
filepath_ct = '../OPC_CT_radiomics_TCIA.csv'
filepath_data = '../OPC_data_clinical.csv'

#make dataframes good
df_ct = pd.read_csv(filepath_ct)
del df_ct[df_ct.columns[0]]
df_ct.set_index("patient", inplace=True)
df_data = pd.read_csv(filepath_data)
df_data.set_index("patient", inplace=True)


In [5]:
basic_information = 0 
shape_features = 23
first_order_features = 38
GLCM_features = 55 
GLDM_features = 79
GLRLM_features = 93 
GLZSM_features = 109 
NGTDM_features = 125

start website

In [ ]:
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, linear_cmap
df_shape = df_ct.iloc[:,shape_features:first_order_features-1]

def render_plot(x, y, z, title):
    #grap group column
    group = df_data.loc[:,z]
    #paste to the frame with the data for the plot
    df = pd.concat([df_shape, group], axis=1)
    #unique factors
    uniq = df[z].unique()

    if len(uniq) == 1:
        color = "purple"
    if len(uniq) <= 2:
        color = factor_cmap(z, ['#1f77b4', '#aec7e8'], uniq)
    if len(uniq) <=20:
        color = factor_cmap(z, bk.palettes.Category20[len(uniq)], uniq)
    else:
        z=""
        color=""

    if y == False:
        y = range(len(df)) 

    p = figure(title = title)
    # p.xaxis.axis_label = x
    # p.yaxis.axis_label = y

    p.scatter(x, y, source=df, color=color, legend_field=z)
    
    return p

In [ ]:
# from bokeh.plotting import figure, show
# from bokeh.transform import factor_cmap
# from bokeh.models import ColumnDataSource, CDSView, GroupFilter
# df_shape = df_ct.iloc[:,shape_features:first_order_features-1]


# shape_cds = ColumnDataSource(df_shape)

# def render_plot(x, y, z, plot_title):
    


In [ ]:
def click_graph_btn(event):
    if not event:
        return    
    body.append(pn.layout.Divider()) 
    body.append(pn.Row(
        pn.Column('# Make a graph', make_graph_input, graph_btn, pn.bind(click_graph_btn, graph_btn)),
        pn.Card(pn.bind(render_plot, x=select_feature1, y=select_feature2, z=select_group, title=input_title), title='Graph')
    ))

    
    

In [ ]:
def change_layout():
    make_graph_input =  pn.Card(select_feature1, select_group, input_title, title="Select your features")
    
    figsingle = pn.bind(render_plot, x=select_feature1, y=False, z=select_group, title=input_title, figsingle=True)
    dashboard.main.append(
     pn.Row(
        pn.Column('# Make a graph', make_graph_input, graph_btn),
        pn.Card(figsingle, title='Graph'))
    )
    

In [ ]:
#options for group
df_nonan = df_data.dropna(axis=1)

#select variables for graph
select_feature1 = pn.widgets.Select(name='Variable for x-axis', options=list(df_shape))
select_feature2 = pn.widgets.Select(name='Variable for y-axis', options=list(df_shape))
select_group = pn.widgets.Select(name='Based on wich group', options=list(df_nonan))
input_title = pn.widgets.TextInput(name='Graph title', placeholder='Enter a string here...')
graph_btn = pn.widgets.Button(name='Make new Graph', button_type='primary')
change_graph_switch = pn.widgets.Switch(name="1 or 2 variables")

#click on buttons
graph_btn.on_click(click_graph_btn)

#switches
change_graph_switch.controls(
    dashboard.main.append(
     pn.Column(pn.Row(
        pn.Column('# Make a graph', pn.Card(select_feature1, select_group, input_title, title="Select your features"), graph_btn),
        pn.Card(pn.bind(render_plot, x=select_feature1, y=False, z=select_group, title=input_title), title='Graph'))
    )
    ))
change_graph_switch.tooltip("### switch between 1 or 2 variables")



#bind objects
figdouble = pn.bind(render_plot, x=select_feature1, y=select_feature2, z=select_group, title=input_title)

#layout for main
make_graph_input =  pn.Card(select_feature1, select_feature2, select_group, input_title, title="Select your features")
body = pn.Column(pn.Row(
        pn.Column(change_graph_switch),
        pn.Column('# Make a graph', make_graph_input, graph_btn),
        pn.Card(figdouble, title='Graph'))
    )

#Instantiate the template with widgets displayed in the sidebar
dashboard = pn.template.BootstrapTemplate(
    title='PlotRadionomics',
    sidebar=[]
)

# Append a layout to the main area, to demonstrate the list-like API
dashboard.main.append(body)

dashboard.show();


voor de show

In [ ]:
#options for group
df_nonan = df_data.dropna(axis=1)

#select variables for graph

select_feature1 = pn.widgets.Select(name='Variable for x-axis', options=list(df_shape))
select_feature2 = pn.widgets.Select(name='Variable for y-axis', options=list(df_shape))
select_group = pn.widgets.Select(name='Based on wich group', options=list(df_nonan))
input_title = pn.widgets.TextInput(name='Graph title', placeholder='Enter a string here...')
graph_btn = pn.widgets.Button(name='Make new Graph', button_type='primary')

#click on buttons
graph_btn.param.watch(click_graph_btn, "value")

#bind objects
fig = pn.bind(render_plot, x=select_feature1, y=select_feature2, z=select_group, title=input_title)


#layout for main
make_graph_input =  pn.Card(select_feature1, select_feature2, select_group, input_title, title="Select your features")
body = pn.Row(
        pn.Column('# Make a graph', make_graph_input, graph_btn),
        pn.Column(fig, title='Graph')
    )

#Instantiate the template with widgets displayed in the sidebar
dashboard = pn.template.BootstrapTemplate(
    title='PlotRadionomics',
    sidebar=[]
)

# Append a layout to the main area, to demonstrate the list-like API
dashboard.main.append(pn.Column(body))

dashboard.show();


In [ ]:
def click_switch(value):
    if value == "moi":
        return pn.Row(pn.Column("#meui", graph_btn))
    if value == "meuk":
        return pn.Row(pn.Column("#moi#moi", change_graph_switch, graph_btn))

In [ ]:
def click_btn(event):
    body.append(pn.Row(pn.Column("#another one", change_graph_switch, graph_btn)))

currently working in this

In [27]:
#imports
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, linear_cmap

#Edit dataset

#main dataset
df_shape = df_ct.iloc[:,shape_features:first_order_features-1]

#options for group choises
df_nonan = df_data.dropna(axis=1)


#Functions

#generate scatter plot with 1 feature
def create_plot_f1(x, z, title):
    if z == "options":
        return
    #grap group column
    group = df_data.loc[:,z]
    #paste to the frame with the data for the plot
    df = pd.concat([df_shape, group], axis=1)
    df.reset_index(level=0, inplace=True)

    #unique factors
    uniq = df[z].unique()

    # Create figure object
    p = figure(title = title)

    #create colormap for different lengths
    if len(uniq) == 1:
        color = '#1f77b4'
    if len(uniq) <= 2:
        color = factor_cmap(z, ['#1f77b4', '#aec7e8'], uniq)
    if len(uniq) <= 20:
        color = factor_cmap(z, bk.palettes.Category20[len(uniq)], uniq)
    if len(uniq) > 20:
        color= linear_cmap(z, palette='Magma256', low=min(uniq), high=max(uniq))
        p.scatter(x, "patient", source=df, color=color, legend_group=z)
        return p

    p.scatter(x, "patient", source=df, color=color, legend_field=z)
    return p

def create_plot_heat(x, y, z, title):
    p = figure(title = title)

    return
    

def add_graph_to_layout(event):
    if not event:
        return
    # make invisible
    if add_graphbox.visible:
        add_graphbox.visible = False
    else:
        #make visibele
        select_graphtype.value = "Options"
        add_graphbox.visible = True


def graphtype_disabledoptions():
    #grab the already disabled graph types
    disablegraphs = []
    alreadydisabled = select_graphtype.disabled_options
    for element in alreadydisabled:
        disablegraphs.append(element)

    #add the new one
    value = select_graphtype.value
    select_graphtype.value = "Options"
    if value != "Options":
        disablegraphs.append(value)
    
    # set value
    select_graphtype.disabled_options = disablegraphs
    return

def show_graphtype_in_body(event):
    if not event:
        return
    
    #add the selected graph type to body
    graphtype = event.new
    if graphtype == "Options":
        return 
    if graphtype == "Scatter(1f)":
        scatter1f = pn.Row(
        pn.Column(pn.layout.Divider(), "### Create your graph", 
                pn.WidgetBox(input_title, autoinput_f1, select_group)
                ),
        pn.Column(fig)
        )
        body.append(scatter1f)
    if graphtype == "Heatmap":
        return
    if graphtype == "Scatter(2f)":
        return
    if graphtype == "Boxplot":
        return

    #make invisible
    add_graphbox.visible = False
    graphtype_disabledoptions()
        
    return

    

dashboard = pn.template.BootstrapTemplate(
    title='PlotRadionomics',
    header_background = "indigo",
    sidebar=[]
)

#Create widgets
## main
btn_add_graph = pn.widgets.Button(name="Add graph", icon="plus", button_type="primary", description="Push to add a another graph", visible=True)
select_graphtype = pn.widgets.Select(name="Choose your graph type", options=["Options", "Scatter(1f)", "Scatter(2f)", "Boxplot", "Heatmap"], disabled_options=[], value="Options")

## Scatter(1f)
autoinput_f1 = pn.widgets.AutocompleteInput(
    name='Filter on feature', options=list(df_shape),
    case_sensitive=False, search_strategy='includes',
    placeholder='Start to write feature')
select_group = pn.widgets.Select(name='Based on wich group', options=list(df_nonan))
input_title = pn.widgets.TextInput(name='Graph title', placeholder='Enter a string here...')

## Heatmap

## Scatter(2f)

## boxplot

#Create body items
add_graphbox = pn.WidgetBox("## Create your graph", select_graphtype)

#Bind widgets
fig = pn.bind(create_plot_f1, x=autoinput_f1, z=select_group, title= input_title)

#watch the widgets
select_graphtype.param.watch(show_graphtype_in_body, "value")
btn_add_graph.param.watch(add_graph_to_layout, "value")



#make invisible
add_graphbox.visible = False


#Display in body
body = pn.Column(pn.Row(pn.Column("## Want to add a graph?", btn_add_graph, add_graphbox)
        ))


dashboard.main.append(body)



dashboard.show();

Launching server at http://localhost:51142


2024-01-08 10:41:58,148 ERROR: panel.reactive - Callback failed for object named "Based on wich group" changing property {'value': 'Primary Treatment'} 
Traceback (most recent call last):
  File "c:\Users\Laris\Documents\repos\p_2.2\DD-radiomics\.venv\Lib\site-packages\panel\reactive.py", line 385, in _process_events
    self.param.update(**self_events)
  File "c:\Users\Laris\Documents\repos\p_2.2\DD-radiomics\.venv\Lib\site-packages\param\parameterized.py", line 2282, in update
    restore = self_._update(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Laris\Documents\repos\p_2.2\DD-radiomics\.venv\Lib\site-packages\param\parameterized.py", line 2322, in _update
    self_._batch_call_watchers()
  File "c:\Users\Laris\Documents\repos\p_2.2\DD-radiomics\.venv\Lib\site-packages\param\parameterized.py", line 2506, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "c:\Users\Laris\Documents\repos\p_2.2\DD-radiomics\.venv\Lib\site-packa

In [10]:
df_shape

,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_10Percentile
patient,,,,,,,,,,,,,,
4,0.540468,29.382158,54.364303,63.976558,61.294372,46.173586,64.876806,36047.000000,37.435671,0.604673,8727.224465,0.242107,36234,21.0
11,0.536580,20.921578,38.990604,44.721360,43.863424,33.120990,46.065171,14326.833333,26.340705,0.687436,4149.716086,0.289646,14404,14.1
12,0.479827,28.477824,59.350228,65.787537,66.850580,49.396356,68.161573,39800.833333,37.641266,0.608044,9271.372824,0.232944,39986,39.0
13,0.481861,22.561287,46.821157,52.430907,38.470768,45.221676,52.896125,18065.250000,30.897470,0.605112,5502.302457,0.304579,18150,2.0
16,0.336308,7.300546,21.707910,20.396078,23.086793,27.018512,27.018512,2032.166667,19.617722,0.644759,1203.352906,0.592153,2078,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.384389,36.016806,93.698856,86.764048,104.651804,63.505905,113.243101,84360.833333,52.907357,0.473039,19664.418897,0.233099,84614,-14.4
597,0.415840,25.254556,60.731494,62.609903,60.926185,40.804412,63.513778,22804.000000,33.599530,0.497842,7811.484381,0.342549,22916,21.0
598,0.468391,28.710600,61.296162,68.622154,69.339743,62.265560,72.166474,31999.166667,52.700886,0.414276,11765.755300,0.367689,32132,-94.5


In [26]:
y

patient
4      29.382158
11     20.921578
12     28.477824
13     22.561287
16      7.300546
         ...    
596    36.016806
597    25.254556
598    28.710600
600    13.332158
602    43.004789
Name: original_shape_LeastAxisLength, Length: 263, dtype: float64

In [24]:
from bokeh.models import LinearColorMapper

p = figure()

x= df_shape["original_shape_Flatness"]
y = df_shape["original_shape_LeastAxisLength"]

p.rect(x=x, y=y)

show(p)

In [ ]:
#imports
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, linear_cmap

#Edit dataset

#main dataset
df_shape = df_ct.iloc[:,shape_features:first_order_features-1]

#options for group choises
df_nonan = df_data.dropna(axis=1)


#Functions

#generate scatter plot with 1 feature
def create_plot_f1(x, z, title):
    if z == "options":
        return
    #grap group column
    group = df_data.loc[:,z]
    #paste to the frame with the data for the plot
    df = pd.concat([df_shape, group], axis=1)
    df.reset_index(level=0, inplace=True)

    #unique factors
    uniq = df[z].unique()

    # Create figure object
    p = figure(title = title)

    #create colormap for different lengths
    if len(uniq) == 1:
        color = '#1f77b4'
    if len(uniq) <= 2:
        color = factor_cmap(z, ['#1f77b4', '#aec7e8'], uniq)
    if len(uniq) <= 20:
        color = factor_cmap(z, bk.palettes.Category20[len(uniq)], uniq)
    if len(uniq) > 20:
        color= linear_cmap(z, palette='Magma256', low=min(uniq), high=max(uniq))
        p.scatter(x, "patient", source=df, color=color, legend_group=z)
        return p

    p.scatter(x, "patient", source=df, color=color, legend_field=z)
    return p

def create_plot_heat(x, y, z, title):
    p = figure(title = title)

    return
    

def add_graph_to_layout(event):
    if not event:
        return
    
    # change staet page to be able to make graphs
    if btn_add_graph.clicks < 1:
        body.remove(body[0])
        body.append(pn.Row(
    pn.Column("# Want to add a graph?", btn_add_graph,
              add_graphbox
            )))
        
    #make visibele the menu to select your graph
    if not add_graphbox.visible:
        start = check_if_disabledgraphs_correct()
        if start:
            add_graphbox.visible = True
            select_graphtype.value = "Options"
        else:
            make_correct_dissabledgraphs



    #make the chosen type graphs not chooseable anymore
    disablegraphs = []
    alreadydisabled = select_graphtype.disabled_options
    for element in alreadydisabled:
        disablegraphs.append(element)
    disablegraphs.append(select_graphtype.value)
    select_graphtype.disabled_options = disablegraphs



def show_graphtype_in_body(event):
    if not event:
        return
    
    #add the selected graph type to body
    graphtype = event.new
    if graphtype == "Options":
        return 
    if graphtype == "Scatter(1f)":
        scatter1f = pn.Row(
        pn.Column("### Create your graph", 
                pn.WidgetBox(input_title, autoinput_f1, select_group)
                ),
        pn.Column(fig)
        )
        body.append(scatter1f)
    if graphtype == "Heatmap":
        return
    if graphtype == "Scatter(2f)":
        return
    if graphtype == "Boxplot":
        return
    #make invisible
    add_graphbox.visible = False
        
    return

def check_if_disabledgraphs_correct():
    start = True
    disabled = select_graphtype.disabled_options

    #checken if the graphtypes truly are in the body
    for graphtype in disabled:
        if graphtype not in body:
            start = False

    return start

def make_correct_dissabledgraphs():
    disabled = select_graphtype.disabled_options

    #remove the items that aint in the body
    for graphtype in disabled:
        if graphtype not in body:
            disabled.remove(graphtype)
    
    select_graphtype.disabled_options = disabled
    return 
    

dashboard = pn.template.BootstrapTemplate(
    title='PlotRadionomics',
    header_background = "indigo",
    sidebar=[]
)

#Create widgets
## main
btn_add_graph = pn.widgets.Button(name="Add graph", icon="plus", button_type="primary", description="Push to add a another graph", visible=True)
select_graphtype = pn.widgets.Select(name="Choose your graph type", options=["Options", "Scatter(1f)", "Scatter(2f)", "Boxplot", "Heatmap"], disabled_options=[], value="Options")

## Scatter(1f)
autoinput_f1 = pn.widgets.AutocompleteInput(
    name='Filter on feature', options=list(df_shape),
    case_sensitive=False, search_strategy='includes',
    placeholder='Start to write feature')
select_group = pn.widgets.Select(name='Based on wich group', options=list(df_nonan))
input_title = pn.widgets.TextInput(name='Graph title', placeholder='Enter a string here...')

## Heatmap

## Scatter(2f)

## boxplot

#Create body items
start_page = pn.FlexBox(pn.Column(
    """# Hello welcome some text to make pretty
    does this help? """,
     "Damn even more to make it magestic",
      "Ahh yes making plots",
      "## Want to add a graph?", btn_add_graph, 
      add_graphbox)
      ,flex_wrap= "wrap-reverse", justify_content="center"
      )
add_graphbox = pn.WidgetBox("## Create your graph", select_graphtype)

#Bind widgets
fig = pn.bind(create_plot_f1, x=autoinput_f1, z=select_group, title= input_title)

#watch the widgets
select_graphtype.param.watch(show_graphtype_in_body, "value")
btn_add_graph.param.watch(add_graph_to_layout, "value")



#make invinsible
add_graphbox.visible = False


#Display in body
body = start_page


dashboard.main.append(body)



dashboard.show();

In [ ]:
btn_add_graph = pn.widgets.Button(name="Add graph", icon="plus", button_type="primary", description="Push to add a another graph", visible=True)

btn_add_graph.param.watch(clickbtn, ["value", "clicks"])

def clickbtn(event):
    if not event:
        return
    
    item = btn_add_graph.clicks
    
    if item < 1:
        print("yes")

    if meuk.visible:
        meuk.visible = False
    else:
        meuk.visible = True

meuk = pn.WidgetBox("## Create your graph", "Choose first the type of graph you want to create", select_graphtype)


body = pn.Row(
    pn.Column("# Want to add a graph?", btn_add_graph,
              meuk
            )
        )

body

BokehModel(combine_events=True, render_bundle={'docs_json': {'b5ed895a-4199-49f0-a7fa-cb6d7024c85a': {'version…

yes


In [ ]:
flex = pn.FlexBox(pn.Column(pn.Row("hoi", "even", "better"), pn.Row("ja")), flex_wrap= "nowrap", align_items="center", justify_content="center")

flex.show()

Launching server at http://localhost:52053


In [ ]:
select_graphtype = pn.widgets.Select(name="Choose your graph type", options=["Options", "Scatter(1f)", "Scatter(2f)", "boxplot", "Heatmap"], disabled_options=["Scatter(1f)"], visible=True)

def select_graphtype_func(event):
    if not event:
        return
    
    value = event.new
    select_graphtype.disabled_options = [value]
    select_graphtype.value = "Options"




select_graphtype.param.watch(select_graphtype_func, 'value')

moi = pn.Row("MOIIIII")
pretty = pn.Row("# pretty pretty")

body = pn.Column(moi, pretty)

if moi not in body:
    print("rip")

per tab werken om een nieuwe grafiek te maken
 op 1 pagina de mogelijkheid te geven om een extra ander soort grafiek te maken
 boxplot
 dus heatmap kan je verschillende factoren selecteren
 een grafiek met 2 verschillende features
 een grafiek waarmee je features per persoon laat zien

 mischien een bepaalde threshold toevoegen
